In [ ]:
# prompt: 코랩에서 한글을 출력할 수 있게 해주세요

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf



In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os, re
from glob import glob
from tqdm import tqdm
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
!wget "https://hycms.hanyang.ac.kr/index.php?module=xn_media_content2013&act=dispXn_media_content2013DownloadContent&content_id=66ef786538650" -O "data.zip"
!unzip  -O cp949 "data.zip" -d "data"

In [ ]:

# %%
"""학습용 데이터셋을 불러옴"""

sdot_data_total = pd.read_csv('data/sdot학습데이터.csv', sep='|', encoding='cp949')
sdot_data_total = sdot_data_total[~sdot_data_total['공'].isnull()]
"""전체 Sdot 평균기온과의 온도차 평균이 높으면 1, 낮으면 -1으로 종속변수 생성"""
sdot_data_total['종속'] = sdot_data_total['온도차이'].apply(lambda x: -1 if x < 0 else 1)

""" 모든 입력변수를 이용한 분석"""
tmp = sdot_data_total

x = np.array(tmp[tmp.columns.drop(['종속', '시리얼번호', '온도차이', '온도비율차이'])].fillna(0).astype('float').values)
y = np.array(tmp['종속'].values)
y = y.reshape(y.shape[0], 1)

In [ ]:
# 데이터를 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# prompt: decision tree 분석
from sklearn.tree import DecisionTreeClassifier

# 의사결정 트리 모델 생성 및 학습
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# 테스트 세트로 예측 수행
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
# prompt: bagging tree 분석

from sklearn.ensemble import BaggingClassifier

# BaggingClassifier 모델 생성 및 학습
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

# 테스트 세트로 예측 수행
y_pred_bagging = bagging_model.predict(X_test)

# 정확도 평가
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print("Bagging Accuracy:", accuracy_bagging)


In [ ]:
# prompt: random forest 분석

from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 생성 및 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 세트로 예측 수행
y_pred_rf = rf_model.predict(X_test)

# 정확도 평가
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)


In [ ]:
# prompt: adaboost dtree 분석

from sklearn.ensemble import AdaBoostClassifier

# AdaBoost 모델 생성 및 학습
ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)
ada_model.fit(X_train, y_train)

# 테스트 세트로 예측 수행
y_pred_ada = ada_model.predict(X_test)

# 정확도 평가
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print("AdaBoost Accuracy:", accuracy_ada)


In [ ]:
# prompt: gbm 분석

from sklearn.ensemble import GradientBoostingClassifier

# GBM 모델 생성 및 학습
gbm_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gbm_model.fit(X_train, y_train)

# 테스트 세트로 예측 수행
y_pred_gbm = gbm_model.predict(X_test)

# 정확도 평가
accuracy_gbm = accuracy_score(y_test, y_pred_gbm)
print("GBM Accuracy:", accuracy_gbm)


In [ ]:
# prompt: xgboost 분석

!pip install xgboost

import xgboost as xgb

# XGBoost 모델 생성 및 학습
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
y_train_xg = y_train.copy()
y_train_xg[np.where(y_train_xg == -1)] = 0
xgb_model.fit(X_train, y_train_xg)

# 테스트 세트로 예측 수행
y_pred_xgb = xgb_model.predict(X_test)

# 정확도 평가
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", accuracy_xgb)


In [ ]:
# prompt: light GBM 분석

!pip install lightgbm

import lightgbm as lgb

# LightGBM 모델 생성 및 학습
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
y_train_lgb = y_train.copy()
y_train_lgb[np.where(y_train_lgb == -1)] = 0
lgb_model.fit(X_train, y_train_lgb)

# 테스트 세트로 예측 수행
y_pred_lgb = lgb_model.predict(X_test)

# 정확도 평가
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("LightGBM Accuracy:", accuracy_lgb)


In [ ]:
# prompt: catboost 분석

!pip install catboost

from catboost import CatBoostClassifier

# CatBoost 모델 생성 및 학습
cat_model = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
y_train_cat = y_train.copy()
y_train_cat[np.where(y_train_cat == -1)] = 0
cat_model.fit(X_train, y_train_cat)

# 테스트 세트로 예측 수행
y_pred_cat = cat_model.predict(X_test)

# 정확도 평가
accuracy_cat = accuracy_score(y_test, y_pred_cat)
print("CatBoost Accuracy:", accuracy_cat)


In [ ]:
# prompt: 트레이닝과 테스트 셋에 대해서 위 모델들의 성능 지표(accuracy_score, precision_score, recall_score, f1_score)를 모두 측정하고 표로 표시

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

models = {
    'Decision Tree': model,
    'Bagging': bagging_model,
    'Random Forest': rf_model,
    'AdaBoost': ada_model,
    'GBM': gbm_model,
    'XGBoost': xgb_model,
    'LightGBM': lgb_model,
    'CatBoost': cat_model
}

results = []
for model_name, model in models.items():
  if model_name == 'XGBoost' or model_name == 'LightGBM' or model_name == 'CatBoost':
    y_train_mod = y_train.copy()
    y_train_mod[np.where(y_train_mod == -1)] = 0
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train_mod, y_pred_train)
    precision_train = precision_score(y_train_mod, y_pred_train, average='macro') # Added average='macro'
    recall_train = recall_score(y_train_mod, y_pred_train, average='macro') # Added average='macro'
    f1_train = f1_score(y_train_mod, y_pred_train, average='macro') # Added average='macro'
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test, average='macro') # Added average='macro'
    recall_test = recall_score(y_test, y_pred_test, average='macro') # Added average='macro'
    f1_test = f1_score(y_test, y_pred_test, average='macro') # Added average='macro'
  else:
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    precision_train = precision_score(y_train, y_pred_train, average='macro') # Added average='macro'
    recall_train = recall_score(y_train, y_pred_train, average='macro') # Added average='macro'
    f1_train = f1_score(y_train, y_pred_train, average='macro') # Added average='macro'
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test, average='macro') # Added average='macro'
    recall_test = recall_score(y_test, y_pred_test, average='macro') # Added average='macro'
    f1_test = f1_score(y_test, y_pred_test, average='macro') # Added average='macro'

  results.append([model_name, accuracy_train, precision_train, recall_train, f1_train, accuracy_test, precision_test, recall_test, f1_test])

df_results = pd.DataFrame(results, columns=['Model', 'Train Accuracy', 'Train Precision', 'Train Recall', 'Train F1', 'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1'])
df_results

In [ ]:
# prompt: df_results의 정확도를 train과 test를 서로 다른 색깔로 차트로 표시해줘

import matplotlib.pyplot as plt

# 모델 이름과 정확도를 추출
models = df_results['Model']
train_accuracy = df_results['Train Accuracy']
test_accuracy = df_results['Test Accuracy']

# 막대 그래프 생성
plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(models))

plt.bar(index, train_accuracy, width=bar_width, label='Train Accuracy', color='blue')
plt.bar([i + bar_width for i in index], test_accuracy, width=bar_width, label='Test Accuracy', color='orange')

# 축 레이블 및 제목 설정
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Train vs. Test Accuracy for Different Models')

# 막대 그래프에 모델 이름 표시
plt.xticks([i + bar_width / 2 for i in index], models, rotation=45, ha='right')

# 범례 표시
plt.legend()

# 그래프 출력
plt.tight_layout()
plt.show()
